# Example: Virtual Detector with Visualization in HyperSpy

An example how to process a pixelated STEM dataset from an HDFS filesystem. 

Prerequisites: 

 * a python3.6 virtualenv with all requirements installed
  * ``pip install hyperspy hyperspy_gui_ipywidgets hdfs3 dask distributed``
  * hdfs3 might need some care as it depends on libhdfs3 which needs to be built first...
 * a running HDFS installation (single node is fine)
 * a dataset imported onto the HDFS filesystem with the ``ingest`` prototype
  * example call: ``$ python ingest.py ~/Data/EMPAD/scan_11_x256_y256.emd experimental/science_data/data test/``
 * ``dask-scheduler`` and ``dask-worker`` running
  * hint: start with something like this, in the processing directory: ``$ OMP_NUM_THREADS=1 OPENBLAS_NUM_THREADS=1 PYTHONPATH=. dask-worker --nprocs=4 --nthreads=1 --host=p70 tcp://<ip-of-scheduler>:8786``
  * we disable threading in OpenBLAS because it would interfere with the dask multiprocessing and because OpenBLAS likes to call ``sched_yield`` for no discernable reason

First, we import our dependencies, including the ``processing`` module from LiberTEM. There may be some warnings about ``hyperspy_gui_traitui`` and so on which can be safely ignored (you do need ``hyperspy_gui_ipywidgets`` though):

In [1]:
%matplotlib nbagg

In [2]:
import processing
import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt

/home/clausen/.virtualenvs/libertem/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Next, we define a helper function for generating masks (stolen from the excellent [fpd_data_processing by Magnus Nord](https://gitlab.com/fast_pixelated_detectors/fpd_data_processing/blob/master/fpd_data_processing/pixelated_stem_tools.py#L59))

In [3]:
def _make_circular_mask(centerX, centerY, imageSizeX, imageSizeY, radius):
    """
    Make a circular mask in a bool array for masking a region in an image.

    Parameters
    ----------
    centreX, centreY : float
        Centre point of the mask.
    imageSizeX, imageSizeY : int
        Size of the image to be masked.
    radius : float
        Radius of the mask.

    Returns
    -------
    Boolean Numpy 2D Array
        Array with the shape (imageSizeX, imageSizeY) with the mask.

    Examples
    --------
    >>> import numpy as np
    >>> import fpd_data_processing.pixelated_stem_tools as pst
    >>> image = np.ones((9, 9))
    >>> mask = pst._make_circular_mask(4, 4, 9, 9, 2)
    >>> image_masked = image*mask
    >>> import matplotlib.pyplot as plt
    >>> cax = plt.imshow(image_masked)
    """
    x, y = np.ogrid[-centerY:imageSizeY-centerY, -centerX:imageSizeX-centerX]
    mask = x*x + y*y <= radius*radius
    return(mask)

Now onto our masks. Many problems in pixelated STEM can be solved by element-wise matrix multiplication and summation of a mask onto each frame, including virtual detector. Here we define some simple radial masks: 

In [ ]:
masks=[
    # random noise for comparison
    np.random.randn(128, 128),
    
    # summation of all pixels:
    np.ones(shape=(128, 128)),
    
    # small radial mask at the center of the frame:
    np.ones(shape=(128, 128)) * _make_circular_mask(
        centerX=64, centerY=64,
        imageSizeX=128, imageSizeY=128, radius=15
    ),
    
    # medium radial mask:
    np.ones(shape=(128, 128)) * _make_circular_mask(
        centerX=64, centerY=64,
        imageSizeX=128, imageSizeY=128, radius=25
    ),
    
    # large radial mask:
    np.ones(shape=(128, 128)) * _make_circular_mask(
        centerX=64, centerY=64,
        imageSizeX=128, imageSizeY=128, radius=50
    ),
    
    # ring, built from a large radial mask minus a smaller radial mask:
    np.ones(shape=(128, 128)) * (
        _make_circular_mask(
            centerX=64, centerY=64,
            imageSizeX=128, imageSizeY=128, radius=50
        ) &
        (~_make_circular_mask(
            centerX=64, centerY=64,
            imageSizeX=128, imageSizeY=128, radius=45
        ))
    ),
]

# visualize all masks:
for mask in masks:
    fig, axes = plt.subplots()
    axes.imshow(mask)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Here we define the ``stackheight``. It says how many frames we process at once. Our dataset has ``128×128`` frames at 8 bytes per pixel, so 8 frames are exactly 1 MB. As my laptop CPU has a L3 cache with a capacity of 8MB, using 4 threads means the working set fits into the L3 cache, even with room for about 32 masks.

In [ ]:
stackheight = 8

Using ``BinaryHDFSDataSet`` we define the input dataset including parameters for connecting to the HDFS namenode, and the ``stackheight`` we set above. With ``ApplyMasksJob`` we define the operation we want to do on the dataset: apply masks. Finally, with ``DaskJobExecutor`` we define the execution environment - we want to use dask in this case, with a scheduler running on the local computer.

In [ ]:
ds = processing.BinaryHDFSDataSet(index_path="test/index.json", host='localhost', port=8020, stackheight=stackheight)
job = processing.ApplyMasksJob(dataset=ds, masks=masks)  # if you want to test with just a single mask, pass masks[-1:] to select just the last one

executor = processing.DaskJobExecutor(scheduler_uri="tcp://localhost:8786")
# print(executor.client.scheduler_info())

Now, we define a HyperSpy signal that will hold our result. The navigation axis will select the result for a specific mask, and the signal axis is the 256×256 result image.

In [ ]:
full_result = hs.signals.Signal2D(np.zeros(shape=(job.maskcount, 256, 256)))
full_result.axes_manager[0].name = "masks"
full_result.axes_manager[1].name = "x"
full_result.axes_manager[2].name = "y"
full_result.plot()
full_result.axes_manager.gui_navigation_sliders()

Now, for kicking off the computation, we tell the ``executor`` to run ``job``. ``run_job`` is a generator and we get the results back as they arrive from the dask scheduler. Each ``result`` is a list of ``ResultTile`` instances, and each ``tile`` represents the result of all masks applied to ``stackheight`` frames. Each ``ResultTile`` knows where in the ``full_result`` it should end up, so we ask it via ``copy_to_result`` to copy its data to the HyperSpy signal data array. We then trigger the ``data_changed`` HyperSpy event, which should enable live updates for results as they come in.

In [ ]:
def run():
    for result in executor.run_job(job):
        for tile in result:
            tile.copy_to_result(full_result.data)
        full_result.events.data_changed.trigger(full_result)
    return tile
%time tile = run()

Nice! With a warm fs cache, LiberTEM can apply 5 masks to the 8GB dataset in about 1.28s (for a single mask, we only take ~800ms).

Let's have a look at one of the result tiles:

In [ ]:
tile

The ``origin`` tells us where in the full dataset this tile originated, and ``shape`` tells us the shape of the *source ``DataTile``* that was processed into this ``ResultTile`` (more on ``DataTile`` later).

In [ ]:
tile.data.shape

Here we can see that the source ``DataTile`` that contained 8 frames with a shape of ``(1, 8, 128, 128)`` had 6 masks applied to it and thus was reduced to a shape of ``(8, 6)``. 

## Data Partitioning

For distributing the computation, we also have to distribute the data. For that, the ``ingest`` script slices the data into partitions. Let's have a look at them:

In [ ]:
[p.slice for p in ds.get_partitions()]

So our dataset is neatly sliced in the first dimension, each partition containing ``16*256 = 4096`` frames. We can also take a look at the index, as it is stored on HDFS:

In [ ]:
ds._index

Now, for efficient computation, each partition itself can be split into even smaller ``DataTile``s:

In [ ]:
p = next(ds.get_partitions())
data_tile = next(p.get_tiles())

In [ ]:
data_tile

So each ``DataTile`` contains ``stackheight`` frames, each of which is 128x128

In [ ]:
len(list(ds.get_partitions()))

In [ ]:
len(list(p.get_tiles()))